### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [14]:
import os
import json
import datetime
import subprocess
import IPython
import IPython.display
#from IPython.core.getipython import get_ipython


lab_prefix="aif-pevo-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style

def stop():
    return

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

#oss=platform.system()
cmdStr= f"az deployment group show --name {deployment_name} -g {resource_group_name} -o json"
cmd=cmdStr.split(' ')
result = subprocess.run(cmd, capture_output=True, text=True, shell=True)

cmd_rtc = result.returncode
cmd_out = result.stdout
cmd_err = result.stderr

#if ( (oss=='Windows' and exit_code != 1) or (oss=='Linux' and exit_code != 0) ): 
if ( cmd_rtc != 0 ): 
        print("Deployment Error", cmd_rtc, deployment_name, resource_group_name)
        print(cmd_err)
        assert(False)

        #raise SystemExit("Stopping the execution of the cell.") 

#deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} -o json   
#print(deployment_stdout)

#deployment = json.loads(deployment_stdout.n)
deployment = json.loads(cmd_out)
for resource in deployment.get("properties").get("outputResources"):
    resource_id = resource.get("id")
    try:
        query = "\"{type:type, name:name, location:location}\""
        resource_stdout = ! az resource show --id {resource_id} --query {query} -o json
        resource = json.loads(resource_stdout.n)
        resource_name = resource.get("name")
        resource_type = resource.get("type")  
        if resource_type == "Microsoft.CognitiveServices/accounts":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az cognitiveservices account delete -g {resource_group_name} -n {resource_name}
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az cognitiveservices account purge -g {resource_group_name} -n {resource_name} -l {resource_location}
            log(delete_stdout, resource_name, "purged")
        elif resource_type == "Microsoft.ApiManagement/service":
            resource_location = "\"" + resource.get("location") + "\""
            delete_stdout = ! az apim delete -n {resource_name} -g {resource_group_name} -y
            log(delete_stdout, resource_name, "deleted")
            delete_stdout = ! az apim deletedservice purge --service-name {resource_name} --location {resource_location}
            log(delete_stdout, resource_name, "purged")
    except:
        print("✌🏻 ", resource_id, " ignored")
delete_stdout = ! az group delete --name {resource_group_name} -y
log(delete_stdout, resource_group_name, "deleted")



Deployment Error 3 function-calling aif-pevo-function-calling
ERROR: (ResourceGroupNotFound) Resource group 'aif-pevo-function-calling' could not be found.
Code: ResourceGroupNotFound
Message: Resource group 'aif-pevo-function-calling' could not be found.



AttributeError: 'IPythonKernel' object has no attribute 'interrupt'